# Tutorial 2 ForkTPS solver
In this tutorial, we learn how to use tensor network approximation parameters with the FTPS solver. In particular, we look at the truncated weight and the maximally allowed bond dimension. As a test case, we use the same model that we also used in the first tutorial

In [ ]:
#%matplotlib notebook
import forktps as ftps
from forktps.solver import DMRGParams, TevoParams

import matplotlib.pyplot as plt

import numpy as np

from forktps.DiscreteBath import *
from forktps.Helpers import *

from triqs.gf import *

The first few steps are the same as in the first tutorial: define a hybridization, discretize it, look at the reconstruction, define the rest of the model, i.e, the interaction and the on-site energy, and initialize the solver.

In [ ]:
#get hybridization
Norbs = 1
gfstruct = [ ['up',[0]], ['dn',[0]] ]

dup = GfReFreq(indices = [0], window = (-3., 3.), n_points =  3001, name = 'up')
ddn = GfReFreq(indices = [0], window = (-3., 3.), n_points =  3001, name = 'dn')

dup['0','0'] << SemiCircular(half_bandwidth = 1.3)
ddn['0','0'] << SemiCircular(half_bandwidth = 2.0)


Delta = BlockGf(name_list = ('up','dn'), block_list = (dup,ddn), make_copies = False)

#discretize bath and look at the reconstruction
Nb = 29 # number of bath sites per spin per orbital
bath = DiscretizeBath(Delta=Delta, Nb=Nb) #bath object storing all relevant information about the bath
eta = 0.1

DeltaDiscrete = bath.reconstructDelta( w = ddn.mesh, eta = eta )

plt.figure()
for i, [name, g] in enumerate(Delta):
    plt.subplot(1,2,i+1)
    w = getX(g.mesh)
    plt.title(name)
    for j in range(g.target_shape[0]):
        plt.plot(w, -Delta[name][j,j].imag.data, label = 'Original')
        plt.plot(w, -DeltaDiscrete[name][j,j].imag.data, label = 'Reconstruction' )
plt.legend()
        
# h0loc and hint
e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill([[0.]]) # put the same 1x1 matrix on both blocks

Hint = ftps.solver_core.HInt(u=0.,j=0., up=0., dd = False)


#finally initialize the solver
S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-3., wmax=3.)
S.b = bath
S.e0 = e0

# How do Tensor Networks approximate states

As with all tensor networks, also the FTPS solver approximates wave functions discarding small Schmidt values of certain bipartitions of the lattice. Note not all possible bipartitions count though, instead the tensor network itself dictates which Schmidt Decompositions can be obtained and are hence approximated.

The Schmidt decomposition of a state $| \psi \rangle$ with respect to two bipartitions $L$ and $R$ is defined as:
$$| \psi \rangle = \sum_{\alpha=1}^{r} s_\alpha | L_\alpha \rangle | R_\alpha \rangle$$


where $| L_\alpha \rangle$ and $| R_\alpha \rangle$ are orthogonal states of the subsystem $L$ and $R$ respectively. $r$ is the so-called Schmidt-Rank and is at most equal to the size of the smallest of the two Hilbert spaces $L$ and $R$. Additionally lets assume $s_\alpha$ are sorted in descending order. 

Tensor Networks approximate states by restricting the Schmidt rank $r$ to some manageable number $m$ on all of the bipartitions accessible by the tensor network. The rest of the states are discarded and the sum of their (squared) Schmidt Values is called the truncated weight $t_w$:

$$t_w = \sum_{\alpha=m+1}^{r} s_\alpha^2$$

and it is the main approximation parameter in any tensor network calculation. $m$ is then the dimension of one of the tensor-indices of the tensor network. Hence taking only the $m$ largest Schmidt values directly restricts the tensor size and with it the numerical effort.<br>
By fixing the truncated weight, one takes as many basis $m$ states as one needs to not discard more than this value. If the truncated weight is chosen too small, this can lead to huge tensors and hence to unacceptably long computation times. To avoid this problem, one often also defines a maximal number of basis states (maximal bond dimension), meaning that no matter how large the truncated weight becomes, one never uses more states than this number. 

#### Obviously, if one uses a maximal bond dimension one should check that the results are converged!

By looking at the FTPS tensor network ([link](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.031013)), we see that the tensors describing the impurities have up to three link indices. Imagining that we use a maximal index dimension of $m$, it is easy to see that these three-link tensors will eventually be the bottleneck of the calculation as $m$ grows. This is because already the size of the impurity tensors is $\sim m^3$ not to speak of the algorithms used to manipulate the tensors. All other tensors having only two link indices are much less problematic.<br>
To be able to deal with this special bottleneck, the FORKTPS solver allows to define truncated weights and maximal bond dimensions for three different kinds of links:

1. Impurity - Impurity links (label 'I' see below)
2. Impurity - Bath links (label 'IB' see below)
3. Bath-Bath links (label 'B' see below)

The maximal Bond dimensions and truncated weights can be set for the time evolution and for DMRG using the two objects DMRGParams and TevoParams.

In [ ]:
# meaning: maximal link dimension of 50 for impurity-impurity links
#          maximal link dimension of 60 for impurity-bath links
#          maximal link dimension of 100 for bath-bath links
#          truncated weight of 1E-09 for impurity-impurity links
#          truncated weight of 1E-10 for impurity-bath links
#          truncated weight of 1E-11 for bath-bath links
paramsDMRG = DMRGParams(maxmI = 50, maxmIB = 60, maxmB=100, twI=1E-9, twIB=1E-10, twB=1E-11)

# tevo params also contain other paramters as for example the time step dt and the number of time steps
paramsTevo = TevoParams(maxmI = 50, maxmIB = 60, maxmB=100, twI=1E-9, twIB=1E-10, twB=1E-11, 
                                    dt = 0.5, time_steps=10) 

The FORKTPS solver allows to define one such approximation for the ground state search (argument *gs_approx*) as well as for the time evolution (argument *tevo_approx*). Generally, time evolutions need larger bond dimensions.

Let us run a couple of calculations with varying accuracy and compare the results (takes a little time):

In [ ]:
# only vary truncated weights; use the same value for time evolution the ground state search for simplicity
tws = [1E-4, 1E-7, 1E-10, 1E-13]
Ggrs = []

# loop over several truncated weights
for tw in tws:
    print(f'Doing tw = {tw}')
    dmrg = DMRGParams(tw=tw) # set tw on all links to the same value
    tevo = TevoParams(tw=tw, dt=0.5, time_steps=10) # set tw on all links to the same value

    S.solve( h_int = Hint, 
             tevo=tevo,
             params_GS = dmrg,
             eta = eta, 
             calc_me = [['up', 0]], # only calculate the Green's function of block 'up' with index 0
             )
    
    # save the greater Gree's function separately
    Ggrs.append(S.G_gr)
 

In [ ]:
# calculate the exact non-interacting greater Green's function for given bath and e0
Ggr_exact = CalcGgr(bath=bath, hloc=e0, mesh = Ggrs[-1].mesh )
t = getX(Ggrs[-1].mesh)
    
    
plt.figure( figsize=(12, 5) )

plt.subplot(1,2,1)
y = Ggr_exact['up'].data[:,0,0]
plt.plot( t, y , label = 'exact', lw = 4, color = 'black' )
for indx, Ggr in enumerate(Ggrs):
    y =Ggr['up'].data[:,0,0].real
    plt.plot( t, y, label ='t_w = ' + str(tws[indx]))

plt.title('Greater Green's function as a function of time')
plt.xlabel('t')
plt.ylabel('$G^>(t)$')
plt.legend()

plt.subplot(1,2,2)
for indx,Ggr in enumerate(Ggrs):
    y =np.abs(Ggr_exact['up'].data[:,0,0] - Ggr['up'].data[:,0,0])
    plt.semilogy( t, y , label = 't_w = ' + str(tws[indx]))

plt.xlabel('t')
plt.ylabel('$|G^>_{ex}(t) - G^>_{FTPS} (t)|$')
plt.title('Numerical error as a function of time')
plt.ylim(1E-10)
plt.legend()

## Discussion
For this simple problem, only the calculation with a truncated weight of $10^{-4}$ gives obviously wrong results with errors of the order of $0.1$. All other Green's functions are acceptable and improve by decreasing the truncated weight. This is of course no surprise, since the calculations were performed with higher and higher accuracy when lowering the truncated weight.

#### Note: There is an additional source of error due to the finite time step $\Delta t$ in the time evolution algorithm (TDVP by default). It should be proportional to $(\Delta t)^2$ but numerical experience tells us that the prefactor seems to be very small (we reach errors of  $10^{-8} << 0.25 = 0.5 ^2 $). 

#### Note, when using TDVP one must be very careful not to reduce the time step too much while keeping the truncation the same. This could result completely wrong Green's functions due to the truncation removing the progress made by the time step. So always check convergence with a bigger time step!

This last point is emphasized in the tutorial: TDVP_LargeTimeSteps